# Face Detection and Recognition
This notebook provides a quick resource for exploring OpenCV's built-in face detection and recognition algorithms, and is inspired by the third section of the following youtube video: https://www.youtube.com/watch?v=oXlwWbU8l2o

Original code for the course can be found on the [github profile](https://github.com/jasmcaus/opencv-course) of the course instructor, Jason Dsouza.

Images of 'famous' people are used as examples, which were downloaded from the Labelled Faces in the Wild (LFW) dataset, available as part of the following Kaggle competition: https://www.kaggle.com/jessicali9530/lfw-dataset

The goal of Face Detection is to identify a rectangular bounding box around each face in an input image. Additional features may be detected as well, e.g. eyes, smile, etc. The goal of Face Recognition is to correctly identify the person based on their facial features. Common practice is to first apply face detection, then apply a facial recognition algorithm to identify the name of the person from features in the detected face image.

## References
- https://github.com/Kaggle/kaggle-api (you can use the api to download the LFW data)
- https://pythonprogramming.net/haar-cascade-object-detection-python-opencv-tutorial/
- https://towardsdatascience.com/a-dog-detector-and-breed-classifier-4feb99e1f852 (also see https://github.com/HenryDashwood/dog_breed_classifier)
- https://keras.io/guides/transfer_learning/
- https://towardsdatascience.com/face-detection-in-2-minutes-using-opencv-python-90f89d7c0f81
- https://www.pyimagesearch.com/2014/07/21/detecting-circles-images-using-opencv-hough-circles/
- https://stackoverflow.com/questions/20801015/recommended-values-for-opencv-detectmultiscale-parameters
- https://docs.opencv.org/4.1.0/d5/d54/group__objdetect.html
- https://www.pyimagesearch.com/2018/02/26/face-detection-with-opencv-and-deep-learning/
- 

### 0. Import Functions

In [1]:
import cv2 as cv
import numpy as np
import os
import random
from opencv_tools import load_frame_gray, show_frame
from opencv_tools import detect_primary_objects, detect_all_objects, detect_image_objects, draw_detected_objects, get_detected_objects

### 1. Set Image Data Parameters
The following parameters are used to load the image data:

`dir_root` = A string defining the base directory that contains subfolders with image files for each person

`people` = A list containing strings that are the names of the "famous" people, and that are the names of the subfolders containing image data. Each name must match the subfolder name exactly after a single modification: spaces, " ", are converted to underscores, "_" (see get_img_path for details). The Analysis in Section 2 provides a list of the people with the most image data that can be used to populate this list

In [2]:
dir_root = "/home/fdpearce/Documents/Projects/data/Images/LFW/lfw-deepfunneled/lfw-deepfunneled"
people = ["George W Bush", "Laura Bush", "Vladimir Putin", "Gloria Macapagal Arroyo", "Arnold Schwarzenegger", "Megawati Sukarnoputri", \
          "Hugo Chavez", "Serena Williams", "Colin Powell", "Junichiro Koizumi", "Jennifer Capriati"]

### 2. Image Availablility Analysis
Print high-level stats on the amount of image data available in the dataset, including 
  1. the number of people with image data
  2. a list of people that have the most images, along with their image count. `num_people_with_most_img` determines how many people to include in the output list. The following parameter is required to perform the analysis:

`num_people_with_most_img` = an integer specifying the number of people to list when displaying the people with the most image data available.

In [3]:
num_people_with_most_img = 25
# Params above, code below
people_folders = os.listdir(dir_root)
num_images_per_folder = [len(os.listdir(os.path.join(dir_root, p))) for p in people_folders]
sort_people_num_img = sorted(zip(people_folders, num_images_per_folder), reverse=True, key=lambda pair: pair[1])
print(f"# of People with Images Available = {len(people_folders)}")
print(f"Top {num_people_with_most_img} People With Most Images, # of Images")
print(*sort_people_num_img[:num_people_with_most_img], sep="\n")

# of People with Images Available = 5749
Top 25 People With Most Images, # of Images
('George_W_Bush', 530)
('Colin_Powell', 236)
('Tony_Blair', 144)
('Donald_Rumsfeld', 121)
('Gerhard_Schroeder', 109)
('Ariel_Sharon', 77)
('Hugo_Chavez', 71)
('Junichiro_Koizumi', 60)
('Jean_Chretien', 55)
('John_Ashcroft', 53)
('Serena_Williams', 52)
('Jacques_Chirac', 52)
('Vladimir_Putin', 49)
('Luiz_Inacio_Lula_da_Silva', 48)
('Gloria_Macapagal_Arroyo', 44)
('Arnold_Schwarzenegger', 42)
('Jennifer_Capriati', 42)
('Laura_Bush', 41)
('Lleyton_Hewitt', 41)
('Hans_Blix', 39)
('Alejandro_Toledo', 39)
('Nestor_Kirchner', 37)
('Andre_Agassi', 36)
('Alvaro_Uribe', 35)
('Megawati_Sukarnoputri', 33)


### 3. Define Functions
These project-specific functions are used for creating training data and visualizing/validating steps in the data processing pipeline

In [4]:
def get_img_path(dir_root, person, img_num, img_ext):
    """Returns the full path to an image by joining the parameters in their input order:
    dir_root = directory root, one level above folders for individual people that contain image files
    person = person the images are of in a specific subfolder, with spaces between first, last, etc
    img_num = a string identifying the individual image of the person
    img_ext = file extension of the image files
    Image format is roughly dir_root/person/person+_+img_num+_+img_ext, see code for details
    """
    person_fname = "_".join(person.split(" "))
    file_name = "_".join([person_fname, img_num])+img_ext
    return os.path.join(dir_root, person_fname, file_name)

def create_training_data(dir_root, people, person_img_max, detect_params, detect_objects=False, detect_type="all", random_seed=0, verbose=False):
    """Create training data from the following inputs:
    dir_root = a string containing base directory with subfolders containing images for each person
    people = a list containing string values, with each string containing a person's name. These are the folder names, EXCEPT for spaces instead of underscores
    person_img_max = an integer specifying the maximum number of training images to return for each person. Set this to a value larger than that
                     maximum # of images available if you want to process all images
    haar_path = a string containing the path to the haar cascade class definition xml file. Files for different features (e.g. face, eyes, etc) can
                be downloaded from the following link: https://github.com/opencv/opencv/blob/master/data/haarcascades/
    detect_params = a dictionary containing two sets of parameters: 1) 'haar_file', a string specifying the full path to the haar cascade
                    xml file to load and 2) 'params' dict to pass to the detectMultiScale method of the haar cascade class. Valid values
                    include scaleFactor (default=1.1), minNeighbors (default=3), and minSize. 
                    The haar_file in 1) can be downloaded from here: https://github.com/opencv/opencv/blob/master/data/haarcascades/
    detect_objects = a boolean-like value (True/False, 1/0, etc) that turns on/off object detection. Set to false to conduct a dry run that checks
                     how many image files will be processed for each person
    detect_type = an optional string specifying the type of detection to perform:
                  "all": runs detect_all_objects, which returns all objects detected from one execution of the haar class detectMultiScale
                  method with the input parameters specified in detect_params. The number of objects detected may vary greatly from image to
                  image for a fixed set of input parameters
                  "primary": runs detect_primary_objects, which performs an iterative process to return a user-specified number of primary objects
                  detected in the input image. Essentially, the minNeighbors parameter is adjusted until the desired number of objects are detected
    random_seed = an integer that determines the order of the random shuffle applied to the list of image files before selecting which to include as training samples
    verbose = a boolean-like value that, when truthy, prints additional details during execution for validation/debugging purposes
    Output is a tuple with two values:
    features = a list containing zero or more lists, with each list containing four values (x, y, w, h) that define the rectangle containing the detected object
    labels = a list containing zero or more integer values, with each int specifying the index to a specific person in the input list of people used for training
    """
    random.seed(random_seed)
    features = []
    labels = []
    for person in people:
        label = people.index(person)
        path = os.path.join(dir_root, "_".join(person.split(" ")))
        img_files = os.listdir(path)
        random.shuffle(img_files)
        img_num = 0
        total_detected = 0
        num_detected = 0
        for img_f in img_files[:min(person_img_max, len(img_files))]:
            img_path = os.path.join(path, img_f)
            img_num += 1
            print(f"Working on Image # {img_num}: {img_f}")
            if detect_objects:
                _, gray = load_frame_gray(img_path, gray_flag=True)
                detected_features, detected_labels = detect_image_objects(gray, detect_params, detect_type=detect_type, label=label, verbose=verbose)
                num_detected = len(detected_features)
                if num_detected:
                    total_detected += num_detected
                    features.extend(detected_features)
                    labels.extend(detected_labels)
        print(f"{img_num} training images with {total_detected} objects identified for {person}")
    return features, labels

def show_person_images(dir_root, person, img_nums, img_ext, object_rectangles, rect_color=(0, 0, 0)):
    """Loop through each each image in the input person's subfolder, whose number is provided in the input list of
    strings, img_nums. Show all objects in the input list of objects, object_rectangles.
    """
    for ind, img_n in enumerate(img_nums):
        img_path = get_img_path(dir_root, person, img_n, img_ext)
        _, gray = load_frame_gray(img_path, gray_flag=True)
        gray = draw_detected_objects(gray, object_rectangles[ind], print_detected=True, frame_to_show=None, rect_color=rect_color)
        show_frame(gray, f"Primary Objects(s) Detected for {person}")

### 4. Face Detection Example

#### 4.1 Load Example Image File
First specify which person you want to analyze from the list, `people`, defined above, by providing a valid index value. Then specify the number of the image (see image files for details), `img_num`, and finally provide the extension for the image file (e.g. ".jpg", ".png", etc), `img_ext`. The code will load the image into a variable, `gray_face`, for subsequent processing.

In [5]:
person = people[5]
img_num = "0027"
img_ext = ".jpg"
# Params above, code below
img_path = get_img_path(dir_root, person, img_num, img_ext)
_, gray_face = load_frame_gray(img_path, gray_flag=True)
gray_face_orig = gray_face.copy()
print(f"Size of Example Grayscale Image = {gray_face.shape}")
show_frame(gray_face, f"Example Image of {person}")

Size of Example Grayscale Image = (250, 250)


#### 4.2 Download Haar Cascade XML File for Face Detection
The first step is to download Haar Cascades xml file for frontal face detection from the following link:

https://github.com/opencv/opencv/blob/master/data/haarcascades/haarcascade_frontalface_default.xml

If you want to follow the code below, then you should also download the eye and smile detection xml files from the haarcascades folder.

#### 4.3 Classic Haar Cascade Detection
##### **Define Detection Parameters**

The dictionary below defines the parameters required to perform object detection (e.g. face, eye, smile) using Haar Cascades. The `haar_file` key defines the full path to the Haar Cascade XML file downloaded in step 4.2 above. The `params` dict can take any key/value pair that is a valid input to the detectMultiScale method of the haar cascade class. The default values provided below should work for most applications, but feel free to change them to optimize performance, etc. See [this link](https://docs.opencv.org/4.1.0/d1/de5/classcv_1_1CascadeClassifier.html) for additional details, such as the default values used in OpenCV for these parameters, additional parameters that can be included, etc.

In [6]:
# Change Haar Cascade detection parameters as necessary
haar_cascade_params = {
    'face': {
        'haar_file': '/home/fdpearce/Documents/Projects/models/haar_cascades/haar_cascade_frontalface.xml',
        'params': {
            'scaleFactor': 1.1,
            'minNeighbors': 4,
            'minSize': (20, 20)
        }
    },
    'eye': {
        'haar_file': '/home/fdpearce/Documents/Projects/models/haar_cascades/haar_cascade_eye.xml',
        'params': {
            'scaleFactor': 1.1,
            'minNeighbors': 4,
            'minSize': (20, 20)
        }
    },
    'smile': {
        'haar_file': '/home/fdpearce/Documents/Projects/models/haar_cascades/haar_cascade_smile.xml',
        'params': {
            'scaleFactor': 1.1,
            'minNeighbors': 8,
            'minSize': (20, 20)
        }     
    }
}

##### **Detect All Faces, Eyes, and Smiles**

In [7]:
detection_params = {
    'objects_to_detect': ['face', 'eye', 'smile'],
    'detector_func': detect_all_objects,
    'verbose': True
}
detected_rects = get_detected_objects(gray_face, haar_cascade_params, **detection_params)

# of Face Objects Detected = 2
# of Eye Objects Detected = 4
# of Smile Objects Detected = 3


##### **Display Results**

In [8]:
print(f"\n# of Faces Found = {len(detected_rects['face'])}")
gray_face = draw_detected_objects(gray_face, detected_rects['face'], print_detected=True, frame_to_show=None, rect_color=(0, 0, 0))
show_frame(gray_face, f"Faces(s) Detected in Image of {person}")
print(f"\n# of Eyes Found = {len(detected_rects['eye'])}")
gray_face = draw_detected_objects(gray_face, detected_rects['eye'], print_detected=True, frame_to_show=None, rect_color=(200, 0, 0))
show_frame(gray_face, f"Faces(s)+Eye(s) Detected in Image of {person}")
print(f"\n# of Smiles Found = {len(detected_rects['smile'])}")
gray_face = draw_detected_objects(gray_face, detected_rects['smile'], print_detected=True, frame_to_show=None, rect_color=(255, 0, 0))
show_frame(gray_face, f"Faces(s)+Eye(s)+Smile(s) Detected in Image of {person}")


# of Faces Found = 2
Object 0 Location: x=7, y=49, w=102, h=102
Object 1 Location: x=74, y=69, w=109, h=109

# of Eyes Found = 4
Object 0 Location: x=129, y=98, w=30, h=30
Object 1 Location: x=95, y=100, w=26, h=26
Object 2 Location: x=33, y=70, w=26, h=26
Object 3 Location: x=71, y=138, w=26, h=26

# of Smiles Found = 3
Object 0 Location: x=121, y=195, w=46, h=23
Object 1 Location: x=100, y=144, w=57, h=29
Object 2 Location: x=33, y=85, w=208, h=104


#### 4.4 Primary Haar Cascade Detection
##### **Define Primary Detection Parameters**

Note that Primary Detection only requires one additional parameter:

`num_primary_obj` = the number of "primary" objects to detect within the image through adjustments to the minNeighbors parameter

In [9]:
# Change Haar Cascade detection parameters as necessary
haar_cascade_params = {
    'face': {
        'haar_file': '/home/fdpearce/Documents/Projects/models/haar_cascades/haar_cascade_frontalface.xml',
        'num_primary_obj': 1,
        'params': {
            'scaleFactor': 1.1,
            'minNeighbors': 4,
            'minSize': (20, 20)
        }
    },
    'eye': {
        'haar_file': '/home/fdpearce/Documents/Projects/models/haar_cascades/haar_cascade_eye.xml',
        'num_primary_obj': 2,
        'params': {
            'scaleFactor': 1.1,
            'minNeighbors': 4,
            'minSize': (20, 20)
        }
    },
    'smile': {
        'haar_file': '/home/fdpearce/Documents/Projects/models/haar_cascades/haar_cascade_smile.xml',
        'num_primary_obj': 1,
        'params': {
            'scaleFactor': 1.1,
            'minNeighbors': 8,
            'minSize': (20, 20)
        }     
    }
}

##### **Detect Primary Faces, Eyes, and Smiles**

In [10]:
gray_face = gray_face_orig
detection_params = {
    'objects_to_detect': ['face', 'eye', 'smile'],
    'detector_func': detect_primary_objects,
    'verbose': True
}
detected_rects = get_detected_objects(gray_face, haar_cascade_params, **detection_params)


Initial # of Face Objects Detected = 2
Iteration # = 1
minNeighbors = 6
Iteration # = 2
minNeighbors = 10
Iteration # = 3
minNeighbors = 16
Iteration # = 4
minNeighbors = 24
Final # of Face Objects Detected = 1

Initial # of Eye Objects Detected = 4
Iteration # = 1
minNeighbors = 6
Final # of Eye Objects Detected = 2

Initial # of Smile Objects Detected = 3
Iteration # = 1
minNeighbors = 10
Iteration # = 2
minNeighbors = 12
Iteration # = 3
minNeighbors = 16
Iteration # = 4
minNeighbors = 22
Final # of Smile Objects Detected = 1


##### **Display Results**

In [11]:
print(f"\n# of Primary Faces Found = {len(detected_rects['face'])}")
gray_face = draw_detected_objects(gray_face, detected_rects['face'], print_detected=True, frame_to_show=None, rect_color=(0, 0, 0))
show_frame(gray_face, f"Faces(s) Detected in Image of {person}")
print(f"\n# of Primary Eyes Found = {len(detected_rects['eye'])}")
gray_face = draw_detected_objects(gray_face, detected_rects['eye'], print_detected=True, frame_to_show=None, rect_color=(200, 0, 0))
show_frame(gray_face, f"Faces(s)+Eye(s) Detected in Image of {person}")
print(f"\n# of Primary Smiles Found = {len(detected_rects['smile'])}")
gray_face = draw_detected_objects(gray_face, detected_rects['smile'], print_detected=True, frame_to_show=None, rect_color=(255, 0, 0))
show_frame(gray_face, f"Faces(s)+Eye(s)+Smile(s) Detected in Image of {person}")


# of Primary Faces Found = 1
Object 0 Location: x=74, y=69, w=109, h=109

# of Primary Eyes Found = 2
Object 0 Location: x=129, y=98, w=30, h=30
Object 1 Location: x=95, y=100, w=26, h=26

# of Primary Smiles Found = 1
Object 0 Location: x=100, y=144, w=57, h=29


#### 4.6 Summary
- **Face Detection**
  - **Classic Detection**: Two faces are detected, and both faces are true positives, but that isn't the desired behavior, as the label only applies to one of the faces. This is a serious challenge when preprocessing the dataset to train a face recognizer: how do we eliminate "secondary" faces so our labels are correctly applied? Note that techniques meant to consolidate multiple detections of the same face (e.g. Non-Maximum Suppression) won't help, and it's likely a more expensive deep learning algorithm would pick these secondary faces up even more consistently than the Haar Cascades used here. For example secondary faces tend to be more rotated away from the camera, out of focus, etc, so Haar Cascades don't detect them as easily.
  - **Primary Detection**: `detect_primary_objects` addresses the secondary faces problem (and false positives), detecting one, primary face within the input image using the same parameters input during Classic Detection in Section 4.3. As we'll see in Section 5, the dataset would contain many mislabeled, inacurrate faces if this issue wasn't resolved, leading to degredation of the facial recognition model's performance, a known issue discussed in the course.
- **Eye Detection**
  - **Classic Detection**: Four eyes are detected, which makes sense given the two faces; however, only three of the eyes are true positives, with the false positive erroneously identifying a curved hand shape as an eye. Overall, the eye detector works pretty well, and this is without limiting its input image to only the region of interest where a face was detected; however, you still can't constrain the output to conform to reason: each face should only have two eyes (or one if it's a profile view). 
  - **Primary Detection**: Only the two eyes on the primary person's face are returned when using `detect_primary_objects` with num_primary_obj = 2, even though the entire image was input, not just the part detected as a face. Impressive! It is worth considering eye detection as an addition data validation step in the training data pipeline. For example, each primary face that is detected must also contain two primary eyes within its bounds in order to be included in the training dataset.
- **Smile Detection**
  - **Classic Detection**: Three smiles were detected: one posible true positive (its definitely a mouth, but one can debate whether it's a smile...), but the other two detections are clearly bad false positives. Overall, the smile detector has pretty good recall but bad precision. Again, better results would be obtained using only the image detected around the faces as input, but even then, I've found smile detection to be the most error prone of the three object types, by far. 
  - **Primary Detection**: Only the correct smile is detected by `detect_primary_objects`, again, even when the whole image is input. As a general rule, primary smile detection requires quite a few iterations to reach a large value for minNeighbors, so the default value may need adjusting, but I've found it to work pretty well so far. It would be interesting to see whether there are patterns in the way the algorithm converges that could be used to identify true vs false positives, probably not, but who knows...

### Face Detection on Training Data
First perform face detection using detection type of 'all', i.e. detection parameters are fixed to those provided at input and all detected objects are returned

In [ ]:
person_img_max = 30
detect_obj = 'face'
detect_type = 'all'
# Params above, code below
features, labels = create_training_data(dir_root, people, person_img_max, haar_cascade_params[detect_obj], detect_objects=True, detect_type=detect_type, verbose=True)
print(f"# of Features = {len(features)}")
print(f"# of Labels = {len(labels)}")

In [ ]:
# Visually inspect example images and the bounding box detected by the face recognition algorithm
# Use the verbose setting above to print out bounding boxes
person = people[0]
img_nums = ["0003", "0019", "0025", "0027", "0031"]
img_ext = ".jpg"
# Megawati Sukarnoputri: 'all' examples where 2 faces detected instead of one
face_rectangles = [[[92, 46, 86, 86], [70, 77, 107, 107]], \
                   [[69, 69, 112, 112], [86, 161, 80, 80]], \
                   [[3, 10, 74, 74], [67, 68, 115, 115]], \
                   [[7, 49, 102, 102], [74, 69, 109, 109]], \
                   [[162, 8, 56, 56], [65, 68, 117, 117]]]
show_person_images(dir_root, person, img_nums, img_ext, face_rectangles, rect_color=(0, 0, 0))

In [ ]:
# Now perform face detection using detection type of 'primary' for comparison
detect_type = 'primary'
haar_obj = haar_cascade_params[detect_obj]
features, labels = create_training_data(dir_root, people, person_img_max, haar_obj, detect_objects=True, detect_type=detect_type, verbose=True)
print(f"# of Features = {len(features)}")
print(f"# of Labels = {len(labels)}")

In [ ]:
face_rectangles = [[[70, 77, 107, 107]], \
                   [[69, 69, 112, 112]], \
                   [[67, 68, 115, 115]], \
                   [[74, 69, 109, 109]], \
                   [[65, 68, 117, 117]]]
show_person_images(dir_root, person, img_nums, img_ext, face_rectangles, rect_color=(0, 0, 0))

### 2. Train Face Recognition Model

In [ ]:
save_recognizer_data = False
face_recognizer = cv.face.LBPHFaceRecognizer_create()

In [ ]:
features = np.array(features, dtype="object")
labels = np.array(labels)

In [ ]:
face_recognizer.train(features, labels)

In [ ]:
if save_recognizer_data:
    face_recognizer.save("face_recognizer.yml")
    np.save("features.npy", features)
    np.save("labels.npy", labels)